스케일이 다른 특성 처리


스케일(Scale)이 다르다는 것은 데이터에 있는 각 특성(feature)의 단위나 범위가 서로 다르다는 의미입니다. 

예를 들어, 어떤 특성은 0~10000 사이의 값을 가지고, 다른 특성은 0~1 사이의 값을 가질 수 있습니다.



왜 스케일을 통일해야 할까요?


모델이 특성들의 스케일을 고려하지 않고 훈련되면, 숫자 값이 큰 특성에 더 큰 중요도를 부여할 수 있습니다. 

마치 키가 180cm인 사람과 몸무게가 80kg인 사람을 비교할 때, '키'의 숫자 180이 '몸무게'의 숫자 80보다 훨씬 중요하다고 착각하는 것과 비슷합니다.


StandardScaler는 이러한 문제를 해결하는 대표적인 방법입니다.

StandardScaler는 모든 특성의 평균을 0, 분산을 1로 맞추어 정규화해줍니다.

이렇게 스케일을 통일하면, 모든 특성이 모델에게 공평하게 다뤄지므로 모델의 성능이 향상되고 훈련이 안정적으로 진행됩니다.


참고: DecisionTreeClassifier와 같은 모델은 스케일에 영향을 받지 않지만, 

LogisticRegression이나 Support Vector Machine 같은 대부분의 다른 모델에서는 스케일링이 필수적입니다.

In [1]:
# 필요한 라이브러리를 불러옵니다.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

# 1. 와인 데이터를 불러와 특성(X)과 타겟(y)을 분리합니다.
df = pd.read_csv("https://bit.ly/wine_csv_data")
X = df[['alcohol', 'sugar', 'pH']]
y = df['class']

# 2. 데이터를 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# 3. 스케일링(Scaling)을 적용합니다.
# StandardScaler 객체를 생성합니다.
ss = StandardScaler()

# 훈련 세트의 평균과 표준편차를 계산하고(fit), 이를 훈련 세트에 적용(transform)합니다.
# '데이터 누수(Data Leakage)'를 막기 위해 fit()은 훈련 세트에서만 합니다.
# 만약 테스트 세트로 fit()을 하면, 모델이 '아직 보지 못한' 테스트 데이터의 정보를
# 미리 알게 되는 '데이터 누수(Data Leakage)'가 발생해 성능이 과장될 수 있습니다.
ss.fit(X_train)

# 훈련 세트와 테스트 세트 모두에 동일한 스케일러를 적용합니다.
# 테스트 세트는 훈련 세트의 통계치를 기반으로 변환됩니다.
X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)

# 4. 스케일이 적용된 데이터로 모델을 훈련합니다.
# DecisionTreeClassifier 모델 객체를 만듭니다.
dt = DecisionTreeClassifier(random_state=42)

# 스케일이 적용된 훈련 세트로 모델을 훈련시킵니다.
dt.fit(X_train_scaled, y_train)

# 5. 모델 훈련 완료 메시지를 출력합니다.
print("--- 스케일이 적용된 데이터로 모델 훈련 완료 ---")

# (추가) 스케일링 전후의 데이터 통계치를 확인해봅니다.
print("\n--- 스케일링 전 훈련 세트의 평균과 표준편차 ---")
print(f"평균: {X_train.mean().to_numpy()}")
print(f"표준편차: {X_train.std().to_numpy()}")

print("\n--- 스케일링 후 훈련 세트의 평균과 표준편차 ---")
print(f"평균: {X_train_scaled.mean(axis=0)}")
print(f"표준편차: {X_train_scaled.std(axis=0)}")


--- 스케일이 적용된 데이터로 모델 훈련 완료 ---

--- 스케일링 전 훈련 세트의 평균과 표준편차 ---
평균: [10.49581897  5.38493432  3.2175821 ]
표준편차: [1.20068084 4.72059384 0.16034537]

--- 스케일링 후 훈련 세트의 평균과 표준편차 ---
평균: [ 2.01991315e-16  1.37091579e-16 -1.92438658e-15]
표준편차: [1. 1. 1.]
